# EcoTravel Agent - Configuração e Teste do Sistema RAG

Este notebook demonstra a configuração detalhada do sistema RAG (Retrieval-Augmented Generation) com estratégias avançadas.

In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
from typing import List, Dict
import time

# Configurar path para imports
sys.path.append('../src')

# Tentar importar dependências RAG
try:
    from sentence_transformers import SentenceTransformer
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import cosine_similarity
    import faiss
    from rank_bm25 import BM25Okapi
    RAG_AVAILABLE = True
except ImportError as e:
    print(f"Algumas dependências RAG não estão disponíveis: {e}")
    print("Execute: pip install sentence-transformers scikit-learn faiss-cpu rank-bm25")
    RAG_AVAILABLE = False

# Configurar visualizações
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

## 1. Configuração do Sistema RAG

In [ ]:
if RAG_AVAILABLE:
    from rag.rag_system import AdvancedRAGSystem
    
    # Inicializar sistema RAG
    print("Inicializando sistema RAG...")
    rag = AdvancedRAGSystem(
        data_path="../data",
        embedding_model="sentence-transformers/all-MiniLM-L6-v2",
        chunk_size=512,
        chunk_overlap=50,
        top_k=10
    )
    
    print("Sistema RAG inicializado com sucesso!")
    print(f"Modelo de embedding: {rag.embedding_model_name}")
    print(f"Tamanho do chunk: {rag.chunk_size}")
    print(f"Sobreposição: {rag.chunk_overlap}")
else:
    print("Sistema RAG não disponível. Instale as dependências necessárias.")

## 2. Carregamento e Processamento de Documentos

In [ ]:
if RAG_AVAILABLE:
    # Carregar documentos
    print("Carregando documentos...")
    start_time = time.time()
    
    documents = rag.load_documents()
    
    load_time = time.time() - start_time
    print(f"Documentos carregados em {load_time:.2f} segundos")
    print(f"Total de documentos: {len(documents)}")
    
    # Analisar documentos carregados
    doc_info = []
    for i, doc in enumerate(documents):
        doc_info.append({
            "index": i,
            "source": doc.metadata.get("source", "unknown"),
            "type": doc.metadata.get("type", "unknown"),
            "length": len(doc.page_content),
            "preview": doc.page_content[:100] + "..."
        })
    
    doc_df = pd.DataFrame(doc_info)
    print("\nInformações dos documentos carregados:")
    print(doc_df[['source', 'type', 'length']].to_string())
else:
    print("Carregamento de documentos não disponível.")

## 3. Criação de Chunks e Análise

In [ ]:
if RAG_AVAILABLE:
    # Criar chunks
    print("Criando chunks semânticos...")
    start_time = time.time()
    
    chunks = rag.create_chunks()
    
    chunk_time = time.time() - start_time
    print(f"Chunks criados em {chunk_time:.2f} segundos")
    print(f"Total de chunks: {len(chunks)}")
    
    # Analisar distribuição de tamanhos dos chunks
    chunk_lengths = [len(chunk.page_content) for chunk in chunks]
    
    print(f"\nEstatísticas dos chunks:")
    print(f"Tamanho médio: {np.mean(chunk_lengths):.1f} caracteres")
    print(f"Tamanho mínimo: {np.min(chunk_lengths)} caracteres")
    print(f"Tamanho máximo: {np.max(chunk_lengths)} caracteres")
    print(f"Desvio padrão: {np.std(chunk_lengths):.1f} caracteres")
    
    # Visualizar distribuição
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.hist(chunk_lengths, bins=20, alpha=0.7, color='skyblue', edgecolor='black')
    plt.title('Distribuição do Tamanho dos Chunks')
    plt.xlabel('Tamanho do Chunk (caracteres)')
    plt.ylabel('Frequência')
    plt.axvline(np.mean(chunk_lengths), color='red', linestyle='--', label=f'Média: {np.mean(chunk_lengths):.0f}')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.boxplot(chunk_lengths)
    plt.title('Box Plot do Tamanho dos Chunks')
    plt.ylabel('Tamanho do Chunk (caracteres)')
    
    plt.tight_layout()
    plt.show()
    
    # Mostrar exemplos de chunks
    print("\nExemplos de chunks criados:")
    for i in range(min(3, len(chunks))):
        print(f"\nChunk {i+1}:")
        print(f"Tamanho: {len(chunks[i].page_content)} caracteres")
        print(f"Fonte: {chunks[i].metadata.get('source', 'unknown')}")
        print(f"Conteúdo: {chunks[i].page_content[:200]}...")
else:
    print("Criação de chunks não disponível.")

## 4. Criação de Embeddings e Índices

In [ ]:
if RAG_AVAILABLE:
    # Criar embeddings
    print("Criando embeddings...")
    start_time = time.time()
    
    embeddings = rag.create_embeddings()
    
    embedding_time = time.time() - start_time
    print(f"Embeddings criados em {embedding_time:.2f} segundos")
    print(f"Shape dos embeddings: {embeddings.shape}")
    print(f"Dimensão dos embeddings: {embeddings.shape[1]}")
    
    # Construir índice FAISS
    print("\nConstruindo índice FAISS...")
    start_time = time.time()
    
    rag.build_faiss_index()
    
    faiss_time = time.time() - start_time
    print(f"Índice FAISS construído em {faiss_time:.2f} segundos")
    print(f"Total de vetores no índice: {rag.faiss_index.ntotal}")
    
    # Construir índice BM25
    print("\nConstruindo índice BM25...")
    start_time = time.time()
    
    rag.build_bm25_index()
    
    bm25_time = time.time() - start_time
    print(f"Índice BM25 construído em {bm25_time:.2f} segundos")
    
    # Resumo dos tempos
    total_time = load_time + chunk_time + embedding_time + faiss_time + bm25_time
    print(f"\n=== Resumo dos Tempos de Construção ===")
    print(f"Carregamento: {load_time:.2f}s ({load_time/total_time*100:.1f}%)")
    print(f"Chunking: {chunk_time:.2f}s ({chunk_time/total_time*100:.1f}%)")
    print(f"Embeddings: {embedding_time:.2f}s ({embedding_time/total_time*100:.1f}%)")
    print(f"FAISS: {faiss_time:.2f}s ({faiss_time/total_time*100:.1f}%)")
    print(f"BM25: {bm25_time:.2f}s ({bm25_time/total_time*100:.1f}%)")
    print(f"Total: {total_time:.2f}s")
else:
    print("Criação de embeddings não disponível.")

## 5. Teste de Diferentes Estratégias de Busca

In [ ]:
if RAG_AVAILABLE:
    # Definir consultas de teste
    test_queries = [
        "Como viajar de São Paulo para Rio de Janeiro de forma sustentável?",
        "Qual o transporte com menor emissão de carbono?",
        "Hotéis sustentáveis no Rio de Janeiro",
        "Atividades ecológicas e ecoturismo",
        "Compensação de carbono para viagens"
    ]
    
    # Testar diferentes métodos de busca
    methods = ["semantic", "lexical", "hybrid"]
    
    search_results = {}
    
    for query in test_queries:
        print(f"\n{'='*60}")
        print(f"CONSULTA: {query}")
        print(f"{'='*60}")
        
        search_results[query] = {}
        
        for method in methods:
            start_time = time.time()
            results = rag.search(query, method=method, k=3, rerank=True)
            search_time = time.time() - start_time
            
            search_results[query][method] = {
                "results": results,
                "time": search_time
            }
            
            print(f"\n--- Método: {method.upper()} ({search_time:.3f}s) ---")
            
            for i, result in enumerate(results[:2], 1):  # Mostrar top 2
                score = result.get('rerank_score', result.get('score', 0))
                print(f"{i}. Score: {score:.3f}")
                print(f"   Conteúdo: {result['content'][:150]}...")
                print(f"   Fonte: {result['metadata'].get('source', 'unknown')}")
else:
    print("Teste de busca não disponível.")

## 6. Análise de Performance dos Métodos

In [ ]:
if RAG_AVAILABLE and 'search_results' in locals():
    # Análise de tempos de busca
    time_analysis = []
    
    for query, methods_data in search_results.items():
        for method, data in methods_data.items():
            time_analysis.append({
                "query": query[:30] + "...",
                "method": method,
                "time": data["time"],
                "num_results": len(data["results"])
            })
    
    time_df = pd.DataFrame(time_analysis)
    
    # Visualizar tempos de busca
    plt.figure(figsize=(15, 5))
    
    # Tempo médio por método
    plt.subplot(1, 3, 1)
    avg_times = time_df.groupby('method')['time'].mean()
    bars = plt.bar(avg_times.index, avg_times.values, color=['skyblue', 'lightcoral', 'lightgreen'])
    plt.title('Tempo Médio de Busca por Método')
    plt.ylabel('Tempo (segundos)')
    plt.xticks(rotation=45)
    
    # Adicionar valores nas barras
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.001,
                 f'{height:.3f}s', ha='center', va='bottom')
    
    # Box plot dos tempos
    plt.subplot(1, 3, 2)
    time_df.boxplot(column='time', by='method', ax=plt.gca())
    plt.title('Distribuição dos Tempos de Busca')
    plt.suptitle('')  # Remove título automático
    
    # Heatmap de tempos por query e método
    plt.subplot(1, 3, 3)
    pivot_times = time_df.pivot(index='query', columns='method', values='time')
    sns.heatmap(pivot_times, annot=True, fmt='.3f', cmap='YlOrRd')
    plt.title('Tempos de Busca por Query e Método')
    plt.xticks(rotation=45)
    plt.yticks(rotation=0)
    
    plt.tight_layout()
    plt.show()
    
    # Estatísticas detalhadas
    print("\n=== Estatísticas de Performance ===")
    print(time_df.groupby('method')['time'].agg(['mean', 'std', 'min', 'max']).round(4))
else:
    print("Análise de performance não disponível.")

## 7. Análise de Qualidade dos Resultados

In [ ]:
if RAG_AVAILABLE and 'search_results' in locals():
    # Análise de diversidade de fontes
    source_analysis = []
    
    for query, methods_data in search_results.items():
        for method, data in methods_data.items():
            sources = set()
            scores = []
            
            for result in data["results"]:
                source = result['metadata'].get('source', 'unknown')
                sources.add(source)
                score = result.get('rerank_score', result.get('score', 0))
                scores.append(score)
            
            source_analysis.append({
                "query": query[:30] + "...",
                "method": method,
                "unique_sources": len(sources),
                "avg_score": np.mean(scores) if scores else 0,
                "max_score": np.max(scores) if scores else 0,
                "score_std": np.std(scores) if scores else 0
            })
    
    source_df = pd.DataFrame(source_analysis)
    
    # Visualizar análise de qualidade
    plt.figure(figsize=(15, 5))
    
    # Diversidade de fontes
    plt.subplot(1, 3, 1)
    avg_sources = source_df.groupby('method')['unique_sources'].mean()
    bars = plt.bar(avg_sources.index, avg_sources.values, color=['skyblue', 'lightcoral', 'lightgreen'])
    plt.title('Diversidade Média de Fontes')
    plt.ylabel('Número de Fontes Únicas')
    plt.xticks(rotation=45)
    
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.02,
                 f'{height:.1f}', ha='center', va='bottom')
    
    # Score médio
    plt.subplot(1, 3, 2)
    avg_scores = source_df.groupby('method')['avg_score'].mean()
    bars = plt.bar(avg_scores.index, avg_scores.values, color=['skyblue', 'lightcoral', 'lightgreen'])
    plt.title('Score Médio dos Resultados')
    plt.ylabel('Score Médio')
    plt.xticks(rotation=45)
    
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                 f'{height:.2f}', ha='center', va='bottom')
    
    # Consistência (desvio padrão dos scores)
    plt.subplot(1, 3, 3)
    avg_std = source_df.groupby('method')['score_std'].mean()
    bars = plt.bar(avg_std.index, avg_std.values, color=['skyblue', 'lightcoral', 'lightgreen'])
    plt.title('Consistência dos Scores\n(menor = mais consistente)')
    plt.ylabel('Desvio Padrão dos Scores')
    plt.xticks(rotation=45)
    
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.001,
                 f'{height:.3f}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()
    
    print("\n=== Análise de Qualidade dos Resultados ===")
    quality_summary = source_df.groupby('method').agg({
        'unique_sources': ['mean', 'std'],
        'avg_score': ['mean', 'std'],
        'max_score': ['mean', 'std']
    }).round(3)
    
    print(quality_summary)
else:
    print("Análise de qualidade não disponível.")

## 8. Exemplo de Contexto Gerado para LLM

In [ ]:
if RAG_AVAILABLE:
    # Demonstrar geração de contexto
    example_query = "Quero viajar de São Paulo para Salvador de forma sustentável. Quais são as melhores opções?"
    
    print(f"CONSULTA: {example_query}")
    print("="*80)
    
    # Gerar contexto usando o método híbrido
    context = rag.get_context_for_query(example_query, max_context_length=1500)
    
    print("CONTEXTO GERADO PELO RAG:")
    print("-"*40)
    print(context)
    print("-"*40)
    print(f"Tamanho do contexto: {len(context)} caracteres")
    
    # Simular prompt completo para o LLM
    system_prompt = """
Você é um assistente especializado em viagens sustentáveis. 
Use as informações do contexto para responder de forma precisa e útil.

CONTEXTO:
{context}

PERGUNTA: {query}

RESPOSTA:"""
    
    full_prompt = system_prompt.format(context=context, query=example_query)
    
    print("\n" + "="*80)
    print("PROMPT COMPLETO PARA O LLM:")
    print("="*80)
    print(full_prompt)
    print("="*80)
    print(f"Tamanho total do prompt: {len(full_prompt)} caracteres")
else:
    print("Geração de contexto não disponível.")

## 9. Benchmark e Métricas do Sistema

In [ ]:
if RAG_AVAILABLE:
    # Criar benchmark de consultas
    benchmark_queries = [
        "transporte sustentável",
        "hotéis ecológicos",
        "emissão carbono avião",
        "ônibus versus trem",
        "atividades locais",
        "compensação carbono",
        "energia solar hotel",
        "mercados locais",
        "certificação LEED",
        "ecoturismo Brasil"
    ]
    
    # Executar benchmark
    benchmark_results = []
    
    print("Executando benchmark do sistema RAG...")
    
    for i, query in enumerate(benchmark_queries):
        print(f"Processando query {i+1}/{len(benchmark_queries)}: {query}")
        
        # Testar busca híbrida
        start_time = time.time()
        results = rag.search(query, method="hybrid", k=5, rerank=True)
        search_time = time.time() - start_time
        
        # Calcular métricas
        if results:
            avg_score = np.mean([r.get('rerank_score', r.get('score', 0)) for r in results])
            max_score = np.max([r.get('rerank_score', r.get('score', 0)) for r in results])
            unique_sources = len(set([r['metadata'].get('source', 'unknown') for r in results]))
        else:
            avg_score = max_score = unique_sources = 0
        
        benchmark_results.append({
            "query": query,
            "search_time": search_time,
            "num_results": len(results),
            "avg_score": avg_score,
            "max_score": max_score,
            "unique_sources": unique_sources
        })
    
    benchmark_df = pd.DataFrame(benchmark_results)
    
    # Estatísticas do benchmark
    print("\n=== RESULTADOS DO BENCHMARK ===")
    print(f"Tempo médio de busca: {benchmark_df['search_time'].mean():.3f}s")
    print(f"Tempo total: {benchmark_df['search_time'].sum():.3f}s")
    print(f"Score médio: {benchmark_df['avg_score'].mean():.3f}")
    print(f"Fontes únicas médias: {benchmark_df['unique_sources'].mean():.1f}")
    print(f"Resultados por query: {benchmark_df['num_results'].mean():.1f}")
    
    # Visualizar benchmark
    plt.figure(figsize=(15, 10))
    
    # Tempo de busca por query
    plt.subplot(2, 2, 1)
    plt.bar(range(len(benchmark_queries)), benchmark_df['search_time'], alpha=0.7)
    plt.title('Tempo de Busca por Query')
    plt.xlabel('Query Index')
    plt.ylabel('Tempo (s)')
    plt.xticks(range(len(benchmark_queries)), [f"Q{i+1}" for i in range(len(benchmark_queries))])
    
    # Score médio por query
    plt.subplot(2, 2, 2)
    plt.bar(range(len(benchmark_queries)), benchmark_df['avg_score'], alpha=0.7, color='green')
    plt.title('Score Médio por Query')
    plt.xlabel('Query Index')
    plt.ylabel('Score Médio')
    plt.xticks(range(len(benchmark_queries)), [f"Q{i+1}" for i in range(len(benchmark_queries))])
    
    # Diversidade de fontes
    plt.subplot(2, 2, 3)
    plt.bar(range(len(benchmark_queries)), benchmark_df['unique_sources'], alpha=0.7, color='orange')
    plt.title('Diversidade de Fontes por Query')
    plt.xlabel('Query Index')
    plt.ylabel('Fontes Únicas')
    plt.xticks(range(len(benchmark_queries)), [f"Q{i+1}" for i in range(len(benchmark_queries))])
    
    # Distribuição dos tempos
    plt.subplot(2, 2, 4)
    plt.hist(benchmark_df['search_time'], bins=10, alpha=0.7, color='purple')
    plt.title('Distribuição dos Tempos de Busca')
    plt.xlabel('Tempo (s)')
    plt.ylabel('Frequência')
    plt.axvline(benchmark_df['search_time'].mean(), color='red', linestyle='--', 
                label=f'Média: {benchmark_df["search_time"].mean():.3f}s')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    # Tabela detalhada
    print("\nResultados detalhados do benchmark:")
    print(benchmark_df.round(3).to_string(index=False))
else:
    print("Benchmark não disponível.")

## 10. Resumo e Conclusões

### Principais Achados:

1. **Performance**: O sistema RAG demonstra tempos de busca consistentes e eficientes
2. **Qualidade**: A busca híbrida combina vantagens semânticas e lexicais
3. **Diversidade**: Resultados abrangem diferentes fontes de conhecimento
4. **Escalabilidade**: Sistema otimizado para base de conhecimento sustentável

### Próximos Passos:
- Integração com LLM para geração de respostas
- Implementação de feedback loop para melhoria contínua
- Expansão da base de conhecimento
- Otimização de parâmetros baseada em métricas